In [ ]:
import os, sys, inspect
data_path = "/data/cxl173430/uncertainty_Related/HENN_Git_VScode/"
sys.path.insert(1, data_path)
# sys.path.insert(1, os.path.join(raps_path, './conformal_classification/'))

from data.tinyImageNet import tinyImageNetVague 

data_dir = '/data/cxl173430/HyperEvidentialNN/TinyImageNet/'
batch_size = 64

mydata = tinyImageNetVague(
            data_dir, 
            num_comp=1, 
            batch_size=batch_size,
            imagenet_hierarchy_path=data_dir,
            duplicate=True)

In [ ]:
print(len(mydata.train_loader.dataset)) # >90200 because of the duplicates
print(len(mydata.valid_loader.dataset))
print(len(mydata.test_loader.dataset))

In [1]:
# Import libraries
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
import sys 
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from random import randint
from datetime import datetime as dt
import time

from sklearn import metrics
import urllib
import zipfile
import csv

import math
import torch
from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.utils.data import Subset, ConcatDataset
from torch.utils.data import random_split, sampler 
from torchvision.utils import make_grid
from torchvision import transforms as T
from torchvision import models, datasets, utils 
from efficientnet_pytorch import EfficientNet

from scipy.optimize import minimize

# DS related 
import dst_pytorch as ds_layer
from dst_pytorch import EfficientNet_DS

In [2]:
parser = argparse.ArgumentParser(description='Process some integers.')

parser.add_argument('-gpu', default=9, type=int, help='The index of the gpu used')
parser.add_argument('-num_comp', default=1, type=int,
                    help='The number of composite elements')
parser.add_argument('-save_dir', default='.', type=str, help='set save directory')
parser.add_argument('-seed', default=123, type=int, help='set random seed')

args = parser.parse_args("")
print(args)

Namespace(gpu=9, num_comp=1, save_dir='.', seed=123)


In [3]:
num_comp = args.num_comp
save_dir = args.save_dir
model_save_dir = os.path.join(save_dir, "models_tiny")
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

if args.seed is not None:
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)
    

# Load Traditional CNN 

In [14]:
import copy
from backbones import EfficientNet_pretrain
num_singles = 200
model = EfficientNet_pretrain(num_singles)

output_folder="/data/cxl173430/uncertainty_Related/HENN_Git_VScode/HyperEvidentialNN_Results/"
saved_spec_dir = "tiny_baseline_DNN"
base_path = os.path.join(output_folder, saved_spec_dir)
print(base_path)

# valid_loader = mydata.valid_loader
test_loader = mydata.test_loader
R = mydata.R
saved_path = os.path.join(base_path, "model_CrossEntropy.pt")
checkpoint = torch.load(saved_path, map_location="cuda:0")
model.load_state_dict(checkpoint["model_state_dict"])

model_best_from_valid = copy.deepcopy(model)
model_best_from_valid.load_state_dict(checkpoint["model_state_dict_best"]) 

# # model after the final epoch
# if args.vaguePred:
#     js_result, prec_recall_f = evaluate_vague_final(model, test_loader, valid_loader, R, device)
# if args.nonVaguePred:
#     acc_nonvague = evaluate_nonvague_final(model, test_loader, device)
# test_result_log(js_result, prec_recall_f, acc_nonvague, False) # bestModel=False

# print(f"### Use the model selected from validation set in Epoch {checkpoint['epoch_best']}:\n")
# if args.vaguePred:
#     js_result, prec_recall_f = evaluate_vague_final(model_best_from_valid, test_loader, valid_loader, R, device)
# if args.nonVaguePred:
#     acc_nonvague = evaluate_nonvague_final(model_best_from_valid, test_loader, device)
# test_result_log(js_result, prec_recall_f, acc_nonvague, True)

Loaded pretrained weights for efficientnet-b3


{'accuracy': tensor(0.7651), 'mean_val_loss': 1.0204317569732666}

# **ECNN training**

## Model architecture 

In [16]:
def fit_ds(model, train_loader, val_loader, lr, epochs, model_save_dir):
    optimizer = optim.Adam(model.parameters(), lr=lr)
#     scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=epochs//2, gamma=0.1)
    global_step = 0
#     lossFun = nn.CrossEntropyLoss()
    criterion = nn.NLLLoss()
    for epoch in range(epochs):
        # Training Phase 
#         print(f" get last lr:{scheduler.get_last_lr()}")
        model.train()
        train_losses = []
        total_correct = 0 
        total_samples = 0 
        for batch in train_loader:
            optimizer.zero_grad()
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)
            logits = model(images)

            #utility layer 
            logits = ds_layer.DM(model.n_classes, 0.9, device=device)(logits) # todo: 
            loss = criterion(torch.log(logits+1e-8), labels)
            
            train_losses.append(loss)
            loss.backward()
            optimizer.step()

            total_correct += numAccurateCE(logits, labels)
            total_samples += len(labels)
            global_step += 1
        scheduler.step() 
        
        mean_train_loss = torch.stack(train_losses).mean().item()
        acc_train = total_correct / total_samples

        # Validation phase
        results = evaluate_cross_entropy(model, val_loader)
        print("Epoch [{}], Mean Training Loss: {:.4f}, Training Accuracy: {:.4f}, Mean Validation Loss: {:.4f}, Validation Accuracy: {:.4f}".format(epoch, mean_train_loss, acc_train, results['mean_val_loss'], results['accuracy']))

        if (epoch + 1) % 1 == 0:
            saved_path = os.path.join(model_save_dir, f'model_ds_whole_{epoch}.pt')
            torch.save(model.state_dict(), saved_path)

    return model 

## Model training 

In [24]:
n_feature_maps = 1536
prototypes=300

model_ds = TinyImagenetEfficientNet_DS(n_feature_maps, num_class, prototypes).to(device)
model_ds

Loaded pretrained weights for efficientnet-b3


TinyImagenetEfficientNet_DS(
  (network): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          40, 10, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          10, 40, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
  

In [25]:
model_prob = TinyImagenetEfficientNet(num_class)
model_prob.load_state_dict(torch.load(saved_path_basic), map_location=args.gpu)

pretrained_dict = model_prob.state_dict()
model_ds_dict = model_ds.state_dict()

pretrained_dict = {k:v for k,v in pretrained_dict.items() if k in model_ds_dict}
# for k, v in pretrained_dict.items():
#     print(k)

model_ds_dict.update(pretrained_dict)
model_ds.load_state_dict(model_ds_dict)

Loaded pretrained weights for efficientnet-b3


<All keys matched successfully>

In [26]:
results_test_ds = evaluate_cross_entropy(model_ds, test_dl)
results_test_ds 

{'accuracy': tensor(0.0005), 'mean_val_loss': 5.303622245788574}

In [27]:
lr_ds = 1e-4
epochs_ds = 30

optimr = "Nadam"

model_save_dir = os.path.join(save_dir, "models_tiny_final-Copy2")
saved_path_ds = os.path.join(model_save_dir, "model_traditional_ds.pt")
saved_path_ds_whole = os.path.join(model_save_dir, "model_traditional_ds_whole.pt")

model_ds = fit_ds(model_ds, train_dl, valid_dl, lr_ds, epochs_ds, model_save_dir, True)
torch.save(model_ds.state_dict(), saved_path_ds)

 get last lr:[0.001]
Epoch [0], Mean Training Loss: 2.3209, Training Accuracy: 0.0048, Mean Validation Loss: 5.3117, Validation Accuracy: 0.0000
 get last lr:[0.001]
Epoch [1], Mean Training Loss: 2.3041, Training Accuracy: 0.0052, Mean Validation Loss: 5.3118, Validation Accuracy: 0.0000
 get last lr:[0.001]
Epoch [2], Mean Training Loss: 2.3031, Training Accuracy: 0.0052, Mean Validation Loss: 5.3118, Validation Accuracy: 0.0000
 get last lr:[0.001]
Epoch [3], Mean Training Loss: 2.3028, Training Accuracy: 0.0050, Mean Validation Loss: 5.3118, Validation Accuracy: 0.0000
 get last lr:[0.001]
Epoch [4], Mean Training Loss: 2.3026, Training Accuracy: 0.0050, Mean Validation Loss: 5.3118, Validation Accuracy: 0.0000
 get last lr:[0.001]
Epoch [5], Mean Training Loss: 2.3026, Training Accuracy: 0.0050, Mean Validation Loss: 5.3118, Validation Accuracy: 0.0000
 get last lr:[0.001]
Epoch [6], Mean Training Loss: 2.3026, Training Accuracy: 0.0050, Mean Validation Loss: 5.3118, Validation Ac

KeyboardInterrupt: 

# **ECNN set Value Prediction**

## generate a utility matrix.

In [112]:
# aim func: cross entropy
def func(x):
    fun=0
    for i in range(len(x)):
        fun += x[i] * math.log10(x[i])
    return fun

#constraint 1: the sum of weights is 1
def cons1(x):
    return sum(x)

#constraint 2: define tolerance to imprecision
def cons2(x):
    tol = 0
    for i in range(len(x)):
        tol += (len(x) -(i+1)) * x[i] / (len(x) - 1)
    return tol


#function for power set
def PowerSetsBinary(items):  
    #generate all combination of N items  
    N = len(items)  
    #enumerate the 2**N possible combinations  
    set_all=[]
    for i in range(2**N):
        combo = []  
        for j in range(N):  
            if(i >> j ) % 2 == 1:  
                combo.append(items[j]) 
        set_all.append(combo)
    return set_all


def average_utility(utility_matrix, inputs, labels, act_set):
    utility = 0
    for i in range(len(inputs)):
        x = inputs[i]
        y = labels[i]
        utility += utility_matrix[x,y]
    average_utility = utility/len(inputs)
    return average_utility


In [ ]:
class_set=list(range(num_class))
# act_set= PowerSetsBinary(class_set)
# act_set.remove(act_set[0])#emptyset is not needed
act_set = R
act_set=sorted(act_set)
print(act_set)
print(len(act_set))

# #label_dict = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer',
# #              5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [ ]:
# #compute the weights g for ordered weighted average aggreagtion
# for j in range(2,(num_class+1)):
#     num_weights = j
#     ini_weights = np.asarray(np.random.rand(num_weights))

# #     name='weight'+str(j)
#     locals()['weight'+str(j)]= np.zeros([5, j])

#     for i in range(5): # why 5? 
#         tol = 0.5 + i * 0.1

#         cons = ({'type': 'eq', 'fun' : lambda x: cons1(x)-1},
#               {'type': 'eq', 'fun' : lambda x: cons2(x)-tol},
#               {'type': 'ineq', 'fun' : lambda x: x-0.00000001}
#             )

#         res = minimize(func, ini_weights, method='SLSQP', options={'disp': True}, constraints=cons)
#         locals()['weight'+str(j)][i] = res.x
#         print(res.x)

In [113]:
#compute the weights g for ordered weighted average aggreagtion
# for j in range(2,(num_class+1)):
j = 2 
num_weights = j
ini_weights = np.asarray(np.random.rand(num_weights))

#     name='weight'+str(j)
locals()['weight'+str(j)]= np.zeros([5, j])

for i in range(5): # why 5? 
    tol = 0.5 + i * 0.1

    cons = ({'type': 'eq', 'fun' : lambda x: cons1(x)-1},
          {'type': 'eq', 'fun' : lambda x: cons2(x)-tol},
          {'type': 'ineq', 'fun' : lambda x: x-0.00000001}
        )

    res = minimize(func, ini_weights, method='SLSQP', options={'disp': True}, constraints=cons)
    locals()['weight'+str(j)][i] = res.x
    print(res.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.30102999566398125
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2
[0.5 0.5]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.29228525323862886
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2
[0.6 0.4]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.2652949955741215
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2
[0.7 0.3]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.2173220112736488
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2
[0.8 0.2]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.14118174150460758
            Ite

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8672697977881468
            Iterations: 13
            Function evaluations: 136
            Gradient evaluations: 13
[0.2544136  0.1959248  0.15074245 0.1158375  0.08907372 0.06879449
 0.05303825 0.04082052 0.03135466]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.7459023319592416
            Iterations: 19
            Function evaluations: 205
            Gradient evaluations: 19
[0.36846462 0.23531913 0.15012562 0.09575836 0.06080199 0.03895117
 0.02497996 0.01548218 0.01011697]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.5367170872124858
            Iterations: 24
            Function evaluations: 263
            Gradient evaluations: 24
[0.55364291 0.24747587 0.11049375 0.04956723 0.02194811 0.00980515
 0.00427679 0.0019655  0.00082468]
Optimization terminated successfully    (Exit mode 0)
    

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.1233008664569368
            Iterations: 24
            Function evaluations: 382
            Gradient evaluations: 24
[0.11511659 0.10609943 0.09777089 0.09010965 0.0830445  0.07652645
 0.07054078 0.0650083  0.0599075  0.05520924 0.05087808 0.04720919
 0.04284035 0.03973904]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.0516196351307954
            Iterations: 28
            Function evaluations: 450
            Gradient evaluations: 28
[0.17607649 0.14768207 0.12404543 0.1039168  0.08734873 0.07326016
 0.06141986 0.05118081 0.04322889 0.03646111 0.03051087 0.02545783
 0.0214722  0.01793876]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.9185227811867992
            Iterations: 28
            Function evaluations: 464
            Gradient evaluations: 28
[0.26727743 0.19662945 0.14487489 0.10644944 0.0

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.7728126804826579
            Iterations: 30
            Function evaluations: 635
            Gradient evaluations: 30
[3.69744470e-01 2.33118332e-01 1.46900568e-01 9.25827091e-02
 5.84915959e-02 3.67802347e-02 2.32573108e-02 1.45358240e-02
 9.19043253e-03 5.75574315e-03 3.67097507e-03 2.30496622e-03
 1.44158813e-03 9.17954094e-04 5.77662577e-04 3.58495626e-04
 2.25827326e-04 1.45310522e-04]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.2787535510090169
            Iterations: 17
            Function evaluations: 353
            Gradient evaluations: 17
[0.05264513 0.05261421 0.05265937 0.05265047 0.05260934 0.05261533
 0.05264653 0.05257039 0.05264041 0.05262329 0.05264729 0.05264686
 0.05258951 0.05268009 0.05264777 0.05264344 0.05263367 0.05262985
 0.05260703]
Optimization terminated successfully    (Exit mode 0)
            Current function val

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8464756429574132
            Iterations: 32
            Function evaluations: 829
            Gradient evaluations: 32
[3.22198786e-01 2.18390570e-01 1.48045086e-01 1.00338042e-01
 6.80448881e-02 4.60976035e-02 3.14234075e-02 2.11575175e-02
 1.43769848e-02 9.59687493e-03 6.60334610e-03 4.56433736e-03
 2.94089325e-03 2.04969470e-03 1.41232869e-03 9.64736686e-04
 6.24724652e-04 4.34366468e-04 2.99524959e-04 2.04735942e-04
 1.38630897e-04 9.29186565e-05]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.3617264891374465
            Iterations: 15
            Function evaluations: 374
            Gradient evaluations: 15
[0.04348578 0.04351893 0.04352303 0.04346738 0.04345747 0.04350591
 0.04335859 0.04344655 0.04347398 0.04345345 0.04352014 0.04347019
 0.04346873 0.04348077 0.04347027 0.04346241 0.04348132 0.04352443
 0.04359221 0.04373313 0.04308948 0.04

...

Iteration limit reached    (Exit mode 9)
            Current function value: -2.0318445597423906
            Iterations: 100
            Function evaluations: 20190
            Gradient evaluations: 100
[0.02383857 0.02345504 0.02296488 0.02227685 0.02210746 0.02126733
 0.020842   0.02033857 0.01975573 0.0196141  0.0189785  0.0185454
 0.01801173 0.0175986  0.01716218 0.01593319 0.01626537 0.01592348
 0.01568495 0.01533799 0.01488139 0.01446411 0.01410936 0.01376403
 0.01353486 0.01320682 0.01292111 0.01252875 0.01220526 0.01192315
 0.01169668 0.01141302 0.01114447 0.01075811 0.01052311 0.01024322
 0.01015584 0.00982996 0.00951561 0.00929617 0.00923714 0.00888441
 0.00874651 0.00841886 0.00822448 0.0080292  0.00791835 0.00775402
 0.00755075 0.00738002 0.00715167 0.00703161 0.00685343 0.00664164
 0.00655883 0.00629386 0.0061784  0.00605798 0.00601377 0.00578651
 0.00563991 0.00552879 0.00531574 0.00520055 0.00519092 0.0049356
 0.0048236  0.00474132 0.0047003  0.00453325 0.00442658 0.0042

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1901685507293585
            Iterations: 78
            Function evaluations: 15795
            Gradient evaluations: 78
[0.01422106 0.01414203 0.01378937 0.0137101  0.01356901 0.01337657
 0.0131591  0.01299001 0.01279048 0.01264075 0.01248093 0.01231553
 0.01215864 0.01187778 0.01183021 0.01166689 0.01152933 0.01135506
 0.01123288 0.01108947 0.01094439 0.01077828 0.01065784 0.01050318
 0.01037219 0.01024384 0.01009952 0.00997981 0.00984608 0.00969248
 0.00956167 0.00946119 0.00931482 0.00913119 0.00904557 0.00892678
 0.00883168 0.00870583 0.00859118 0.00847827 0.00836333 0.00824398
 0.00814488 0.00803664 0.00794436 0.0078415  0.00772123 0.00763924
 0.00752543 0.00743617 0.00734979 0.00724117 0.00713171 0.00702425
 0.00693185 0.00686198 0.00676558 0.00668169 0.00660519 0.0064998
 0.00642149 0.006316   0.00625595 0.00617214 0.00608239 0.00598036
 0.005942   0.00584151 0.00574166 0.00567349 0.005

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2749027760891023
            Iterations: 48
            Function evaluations: 9742
            Gradient evaluations: 48
[0.00863839 0.00862649 0.00854602 0.00848354 0.0083584  0.00839064
 0.00835478 0.00822226 0.0082703  0.0084973  0.00805096 0.00806006
 0.00797197 0.00805371 0.00790322 0.00787146 0.00783518 0.00779887
 0.00781986 0.0077061  0.00762593 0.00760699 0.00751001 0.00747806
 0.00739951 0.00739964 0.0073542  0.0073208  0.00731896 0.00723937
 0.00719466 0.00719133 0.00711895 0.00712618 0.00704043 0.00695101
 0.00695762 0.0068895  0.00681895 0.00678602 0.00677234 0.00671036
 0.00665502 0.006618   0.00657429 0.00656035 0.00650446 0.00648912
 0.0064321  0.00640172 0.00636264 0.00632521 0.0062464  0.00624359
 0.00621085 0.00615137 0.00616002 0.00608991 0.0060365  0.0060021
 0.00599084 0.00589313 0.00589844 0.00588337 0.00584205 0.00578792
 0.00575431 0.00572104 0.0056843  0.00569522 0.0056

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3031389001023617
            Iterations: 29
            Function evaluations: 5908
            Gradient evaluations: 29
[0.00498934 0.00503233 0.00497029 0.00495936 0.00498343 0.00499275
 0.00497943 0.00499331 0.00499339 0.00501557 0.00495318 0.00497757
 0.00497937 0.00499028 0.00497307 0.00496923 0.0049705  0.00496766
 0.00497641 0.0049537  0.00498116 0.00501056 0.00496953 0.00500701
 0.00498802 0.00496722 0.00496858 0.00497257 0.00497296 0.004992
 0.00495334 0.00488669 0.00498152 0.00499688 0.00499959 0.00499133
 0.00498505 0.00497192 0.00500993 0.00491756 0.00499671 0.00495358
 0.00488995 0.00494119 0.00499181 0.00495389 0.00497443 0.00495809
 0.00495429 0.00501045 0.00497889 0.00499619 0.00498163 0.00500066
 0.00502138 0.00498899 0.00498379 0.00497959 0.00485214 0.00496248
 0.00500374 0.00497768 0.00497535 0.00496055 0.00497277 0.00497731
 0.00503072 0.00491091 0.00507879 0.00499345 0.00498

Iteration limit reached    (Exit mode 9)
            Current function value: -1.7453552950089921
            Iterations: 100
            Function evaluations: 20633
            Gradient evaluations: 100
[4.73001181e-02 4.41515914e-02 4.29603757e-02 4.07768398e-02
 3.99716738e-02 3.67603527e-02 3.55698851e-02 3.39899095e-02
 3.23967652e-02 3.06982605e-02 2.99669683e-02 2.77654857e-02
 2.74164992e-02 2.68183131e-02 2.45748661e-02 2.28167054e-02
 2.14038810e-02 2.02680542e-02 1.94254578e-02 1.87776763e-02
 1.67519355e-02 1.72716974e-02 1.62145717e-02 1.58428260e-02
 1.46773560e-02 1.41092877e-02 1.32686648e-02 1.29973847e-02
 1.22757874e-02 1.16553450e-02 1.09645257e-02 1.06876705e-02
 1.01887223e-02 9.67023603e-03 8.99974305e-03 8.68618146e-03
 8.36217172e-03 8.01867612e-03 7.72582043e-03 7.39044117e-03
 6.96884230e-03 6.58221231e-03 6.40209864e-03 5.91557426e-03
 5.57403348e-03 5.21754498e-03 5.15330031e-03 4.61714929e-03
 4.58972382e-03 4.47989532e-03 4.10574639e-03 4.10012782e-03
 3.4

In [116]:
utility_matrix = np.zeros([len(act_set), len(class_set)])
tol_i = 3 
#tol_i = 0 with tol=0.5, tol_i = 1 with tol=0.6, tol_i = 2 with tol=0.7, tol_i = 3 with tol=0.8, tol_i = 4 with tol=0.9
for i in range(len(act_set)):
    intersec = class_set and act_set[i]
    if len(intersec) == 1:
        utility_matrix[i, intersec] = 1
  
    else:
        for j in range(len(intersec)):
            utility_matrix[i, intersec[j]] = locals()['weight'+str(len(intersec))][tol_i, 0]
print(utility_matrix)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [117]:
utility_matrix.shape  

201

In [118]:
class DM_test(nn.Module):
    '''
    '''
    def __init__(self, num_class, num_set, nu):
        super(DM_test, self).__init__()
        self.num_class = num_class 
        self.nu = nu 
        self.utility_matrix = nn.Linear(in_features = num_class, out_features = num_set, bias=False).weight 
    
    def forward(self, inputs):
        for i in range(len(self.utility_matrix)): 
            if i == 0:
                precise = torch.mul(inputs[:, 0: self.num_class], self.utility_matrix[i])
                precise = torch.sum(precise, dim=-1, keepdim=True) #dim=0, 1 ?
                
                omega_1 = torch.mul(inputs[:, -1], torch.max(self.utility_matrix[i]))
                omega_2 = torch.mul(inputs[:, -1], torch.min(self.utility_matrix[i]))
                
                omega = torch.unsqueeze(self.nu*omega_1+(1-self.nu)*omega_2, -1)
                omega = omega.type(torch.float32)
                utility = precise + omega
                
            if i >= 1:
                precise = torch.mul(inputs[:, 0: self.num_class], self.utility_matrix[i])
                precise = torch.sum(precise, dim=-1, keepdim=True) #dim=0, 1 ?
                
                omega_1 = torch.mul(inputs[:, -1], torch.max(self.utility_matrix[i]))
                omega_2 = torch.mul(inputs[:, -1], torch.min(self.utility_matrix[i]))
                
                omega = torch.unsqueeze(self.nu*omega_1+(1-self.nu)*omega_2, -1)
                omega = omega.type(torch.float32)
                utility_i = precise + omega
                utility = torch.cat([utility, utility_i], -1)
                
        return utility 
    


In [ ]:
def calculate_metrics(outputs, labels, act_set):
    correct_vague = 0.0
    correct_nonvague = 0.0
    vague_total = 0
    nonvague_total = 0
    
    # Get the predicted labels
    pred_label_ids = torch.argmax(outputs, dim=1)
    imprecise_results =[]
    for i in range(len(labels)):
        act_local = pred_label_ids[i]
        set_valued_results = act_set[act_local]
        imprecise_results.append(set_valued_results)
        
        ground_truth_set = set(act_set[labels[i].item()]) ##todo: correct this  
        
        predicted_set = set(set_valued_results)
        inter_set = predicted_set.intersection(ground_truth_set)
        union_set = predicted_set.union(ground_truth_set)
        rate = float(len(inter_set)/len(union_set))
        
        if len(predicted_set) == 1:
            correct_nonvague += rate
            nonvague_total += 1
        else:
            correct_vague += rate 
            vague_total += 1
      
    return [correct_nonvague, correct_vague, nonvague_total, vague_total]


@torch.no_grad()
def evaluate_ds(model, val_loader, num_class, num_set, nu, utility_matrix):
    dmTest = DM_test(num_class, num_set, nu).to(device)  
    dmTest.weight = nn.Parameter(torch.tensor(utility_matrix).T)
    
    model.eval()
    correct_vague = 0.0
    correct_nonvague = 0.0
    vague_total = 0
    nonvague_total = 0 
    
    lossFun = nn.CrossEntropyLoss()
    results_all = {
      
        'vague_nonvague': []
    }
    total_correct = 0
    total_samples = 0
   
    for batch in val_loader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)
        ds_normalize = model(images)
        outputs = dmTest(ds_normalize)
        
        results = calculate_metrics(outputs, labels, act_set)
        correct_nonvague += results[0]
        correct_vague += results[1]
        nonvague_total += results[2]
        vague_total += results[3]
    
   
    results_all['vague_nonvague'] = [correct_vague, correct_nonvague, vague_total, nonvague_total]
#     results['mean_val_loss'] = torch.stack(val_losses).mean().item()
#     results['vague_nonvague'] = [correct_vague, correct_nonvague, vague_total, nonvague_total]
    
    return results_all


## Build an Evidential NN 

In [119]:
model_ds_set = TinyImagenetEfficientNet_DS(n_feature_maps, num_class, prototypes).to(device)
model_ds_set.load_state_dict(torch.load(saved_path_ds_whole))

num_set = 201 
results_finals = evaluate_ds(model_ds_set, test_dl, num_class, num_set, 0.9, utility_matrix)

Loaded pretrained weights for efficientnet-b3


In [120]:
results_f = results_finals['vague_nonvague']
avg_js_vague = results_f[0] / (results_f[2]+1e-5)
print(f"Average Jaccard Similarity Vague Classes:{avg_js_vague}")

avg_js_nonvague = results_f[1] / (results_f[3]+1e-5)
print(f"Average Jaccard Similarity Nonvague Classes:{avg_js_nonvague}")

overall_acc = (results_f[0] + results_f[1])/(results_f[2] + results_f[3] + 1e-5)
print(f"Overall Accuracy: {overall_acc}")

Average Jaccard Similarity Vague Classes:0.0
Average Jaccard Similarity Nonvague Classes:0.0049999999950000006
Overall Accuracy: 0.0049999999950000006
